In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Median

In [2]:
df = pd.read_excel("FX_Returns.xlsx", index_col="Date",parse_dates=True)

In [3]:
returns=df

In [4]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.4/983.4 kB 17.8 MB/s eta 0:00:00


In [5]:
from arch import arch_model

In [6]:
df1= pd.read_excel("FX Daily Realized Covariance (Neural).xlsx", sheet_name= "Vol", index_col="Date",parse_dates=True)


In [7]:
df1.iloc[int(len(df1)*0.7)]

AUD_JPY     0.017456
EUR_USD     0.015948
EUR_JPY     0.025621
AUD_USD     0.008161
GBP_USD     0.004309
USD_CAD     0.002223
Name: 2015-06-28 00:00:00, dtype: float64

In [8]:
forecasts1=np.zeros([int(len(df1)*0.3),len(df.columns)])
test_size = int(len(df1)*0.3)
for i in range(len(df.columns)):
  am = arch_model(returns[df.columns[i]], vol="garch", p=1, o=0, q=1, dist="Normal")
  res = am.fit(update_freq=5,last_obs="2015-06-28")
  rolling_predictions = []
  for j in range(int(test_size/1)):
    train = returns[:-(test_size-j*1)]
    model = arch_model(train[df.columns[i]], p=1, o=0, q=1)
    model_fit = model.fit(disp='off')
    pred = model_fit.forecast(horizon=1)
    rolling_predictions.append(np.sqrt(pred.variance.values[-1,:]))
  rolling_predictions=np.sqrt(rolling_predictions)**2
  rolling_predictions=rolling_predictions.reshape(forecasts1.shape[0])
  forecasts1[:,i]=rolling_predictions/100

Iteration:      5,   Func. Count:     35,   Neg. LLF: 3824.98412567197
Iteration:     10,   Func. Count:     65,   Neg. LLF: 3788.424389335541
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3788.4243301630045
            Iterations: 12
            Function evaluations: 74
            Gradient evaluations: 12
Iteration:      5,   Func. Count:     34,   Neg. LLF: 18215.16493910453
Iteration:     10,   Func. Count:     68,   Neg. LLF: 2581.034406992253
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2580.141405542023
            Iterations: 14
            Function evaluations: 89
            Gradient evaluations: 14
Iteration:      5,   Func. Count:     33,   Neg. LLF: 3255.2582550504303
Iteration:     10,   Func. Count:     63,   Neg. LLF: 3237.108479012119
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3237.108465173156
            Iterations: 12
            Fu

In [9]:
df2=df1.iloc[int(len(df1)*0.7)+1:len(df1)]

In [10]:
for i in range(len(df2.columns)):
  df2[df2.columns[i]]=forecasts1[:,i]

<ipython-input-10-ae522ed45476>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[df2.columns[i]]=forecasts1[:,i]


In [11]:
df2.to_excel("GARCH(1,1)-Norm forecasts.xlsx")